In [2]:
from ckonlpy.tag import Twitter
import pickle
import re
import sys
from gensim.models import TfidfModel
from gensim.models import Word2Vec
from gensim.corpora import Dictionary
from pprint import pprint
from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import mglearn
from pprint import pprint
from ckonlpy.tag import Twitter
import numpy as np
import gc
import copy

class Social_analysis():
    
    non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

    def __init__(self):
        self.twitter = Twitter()
        
    def pickle_to_table(self, filename):
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        data = data[1:]
        for idx, i in enumerate(data):
            data[idx][2] = i[2].replace('#', ' ').translate(self.non_bmp_map)
            data[idx][3] = '/'.join(i[3])
            data[idx][4] = '/'.join(i[4])
        self.raw_data = np.array(data)
  
    def hashtags_split(self, hashtags):        
        hashtags_split = []
        for i in hashtags:
            hashtags_split.append(i.split('/'))
        
        hashtags_list = []
        
        for i in hashtags_split:
            temp = []
            for j in i:
                if self.isHangul(j):
                    t_hashtags = j.translate(self.non_bmp_map)
                    temp.append(t_hashtags)
            hashtags_list.append(temp)
        self.hashtags_list = hashtags_list
        
        return hashtags_list
                    
    def add_keyword_dic(self, keyword_list, tag='Noun'):
        for i in keyword_list:
            if type(i) == tuple:
                self.twitter.add_dictionary(i[0], i[1])
            else:
                self.twitter.add_dictionary(i, tag)
        
    def morph_pos(self, text_list, exception_list = ['맛', '밥', '물', '몸']):
        
        morph_list = []
        noun_list = []
        adj_list = []
        verb_list = []
        
        for j in text_list:
            parsed = self.twitter.pos(j)
            temp = []
            n_temp = []
            adj_temp = []
            verb_temp = []
            
            for i in parsed:
                if self.isHangul(i[0]):
                    if ((len(i[0]) > 1) or (i[0] in exception_list)):
                        temp.append(i)
                        if i[1] == 'Noun':
                            n_temp.append(i[0])
                        elif i[1] == 'Verb':
                            adj_temp.append(i[0])
                        elif i[1] == 'Adjective':
                            verb_temp.append(i[0])
                    else:
                        print('{} 제외'.format(i[0]))
                else: print('{} 한글이 아님.'.format(i[0]))
            

            morph_list.append(temp)
            noun_list.append(n_temp)
            adj_list.append(adj_temp)
            verb_list.append(verb_temp)

        nav_list = noun_list + adj_list + verb_list

        return morph_list, nav_list, noun_list, adj_list, verb_list


    
    def pos_extractor(self, parsed):
        for i in parsed:            
            temp = []
            n_temp = []
            adj_temp = []
            verb_temp = []
            if self.isHangul(i[0]):
                if ((len(i[0]) > 1) or (i[0] in exception_list)):
                    temp.append(i)
                    if i[1] == 'Noun':
                        n_temp.append(i[0])
                    elif i[1] == 'Verb':
                        adj_temp.append(i[0])
                    elif i[1] == 'Adjective':
                        verb_temp.append(i[0])
                else:
                    print('{} 제외'.format(i[0]))
            else: print('{} 한글이 아님.'.format(i[0]))
            

            morph_list.append(temp)
            noun_list.append(n_temp)
            adj_list.append(adj_temp)
            verb_list.append(verb_temp)

        nav_list = noun_list + adj_list + verb_list

        return morph_list, nav_list, noun_list, adj_list, verb_list

    
    
    def merge_list(self, tokenized_list):
        return [j for i in tokenized_list for j in i]

    
    def join_list(self, tokenized_list):
        joined_list = []
        for idx, i in enumerate(tokenized_list):
            joined_list.append(" ".join(i))
        return joined_list
 
    def split_list(self, untokenized_list):
        hashtag_splited = []
        for idx, i in enumerate(untokenized):
            hashtag_splited.append(i.split('/'))
            return hastag_splited

    def word_substitute(self, dataset, sublist):
        dataset = copy.deepcopy(dataset)
        sub_book = dict()
        for i in sublist:
            for j in i['sub_words']:
                sub_book[j] = i['main']
        gc.collect()
        for n, i in enumerate(dataset):
            dataset[n] = [sub_book.get(item,item) for item in i]

        del sub_book
        gc.collect()

        return dataset
    
    def word_delete(self, dataset, del_list):
        dataset = copy.deepcopy(dataset)

        for n, line in enumerate(dataset):
             dataset[n] = [i for i in line if i not in del_list]

        return dataset

    
    def isHangul(self, text):
        encText = text
        hanCount = len(re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', encText))
        return hanCount > 0

    

class SB_Word2Vec():    
    
    def __init__(self, morph_list):
        self.dct = Dictionary(morph_list)
        self.corpus = [self.dct.doc2bow(line) for line in morph_list]
        self.build_Word2Vec(morph_list)
    
    def make_Word2Vec(self, morph_list, size=50, window=2, min_count=10, iteration=100):
        self.em = Word2Vec(morph_list, size=size, window=window, min_count=min_count, iter=iteration)
        self.em_vocab = list(self.em.wv.vocab.keys())
        self.em_vocab_dic = {word:idx for idx, word in enumerate(self.em_vocab)}

    def make_Word2Sen_matrix(self): 
        vocab_size = len(self.em_vocab)
        self.sen_matrix = np.zeros((len(self.corpus), vocab_size))
        for idx, row in enumerate(self.sen_matrix):
            for idx2, frequency in self.corpus[idx]:
                    if self.dct[idx2] in self.em_vocab:
                        self.sen_matrix[idx][self.em_vocab_dic[self.dct[idx2]]] = frequency                
        self.sim_matrix = np.zeros((vocab_size, vocab_size))
        for idx, w1 in enumerate(self.em_vocab):
            for idx2, w2 in enumerate(self.em_vocab):
                self.sim_matrix[idx][idx2] =  self.em.wv.similarity(w1, w2)

        self.word2sen_matrix = np.dot(self.sim_matrix, np.transpose(self.sen_matrix))

        return self.word2sen_matrix

    def get_sim_sen(self, keyword, main_text, number=1):
        self.sim_sen_index = np.argsort(self.word2sen_matrix[self.em_vocab_dic[keyword]])
        self.most_sim_sen_index = np.argmax(self.word2sen_matrix[self.em_vocab_dic[keyword]])
        index_list = self.sim_sen_index.reshape((-1,)).tolist()
        index_list.reverse()
        
        for idx, i in enumerate(index_list[:number]):
            print(str(idx + 1))
            print(main_text[i])
        return index_list
    
    def build_Word2Vec(self, morph_list):
        self.make_Word2Vec(morph_list)
        self.make_Word2Sen_matrix()
        
        
class SB_LDA():

    def make_lda(self, morph_joined, ntopic=10, learning_method='batch', max_iter=25, random_state=0, n_words=20):        
        self.vect = CountVectorizer(max_features=10000, max_df=.15)
        self.X = self.vect.fit_transform(morph_joined)
        self.lda = LatentDirichletAllocation(n_components=ntopic, learning_method=learning_method, max_iter=max_iter, random_state=random_state)
        self.document_topics = self.lda.fit_transform(self.X)
        self.sorting = np.argsort(self.lda.components_, axis=1)[:, ::-1]
        self.feature_names = np.array(self.vect.get_feature_names())
        mglearn.tools.print_topics(topics=range(ntopic), feature_names=self.feature_names, sorting=self.sorting, topics_per_chunk=5, n_words=n_words)

    def related_doc(self, main_text_list, topic_index, number=10):
        category = np.argsort(self.document_topics[:, topic_index])[::-1]
        related_docs = []
        for i in category[:number]:
            print(i)
            print(main_text_list[i] + ".\n")
            related_docs.append((i, main_text_list[i]))
        return related_docs

class SB_Tfidf():    
    
    def __init__(self, list_morph_merged):
        self.list_morph_merged = list_morph_merged
        self.dct = Dictionary(self.list_morph_merged)
        self.corpus = [self.dct.doc2bow(line) for line in self.list_morph_merged]

    def get_tfidf(self):       
        self.model = TfidfModel(self.corpus)
        self.tfidf = []
        for i in self.corpus:
             self.tfidf.append(sorted(self.model[i], key = lambda x: x[1], reverse=True))
        self.tfidf_hangul = []
        for idx1, i in enumerate(self.tfidf):
            self.tfidf_hangul.append([(self.dct[j[0]], j[1]) for j in i])        
        
        return self.tfidf_hangul
    
def frequency(merged):
    word_count = Counter(merged)
    word_count2 = []
    for i in word_count:
        word_count2.append((i, word_count[i]))
    word_count2 = sorted(word_count2, key=lambda x: x[1], reverse = True)
    return word_count2

C:\Users\abys\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
intake = Social_analysis()

In [4]:
labnosh = Social_analysis()

## DB에서 불러온 데이터를 Social_analysis 인스턴스에 등록해도  되고,  그냥 그 데이터로 앞으로 진행해도 된다. 일단 튜토리얼은 pickle 데이터로 진행하기 때문에 아래 함수를 실행하면 intake.row_data 에 저장이 된다. 이 놈은 DB에 들어간 놈이 비슷하게 생긴 놈이다.

In [5]:
intake.pickle_to_table('Data/intake_list.txt')

In [6]:
labnosh.pickle_to_table('Data/labnosh_list.txt')

## 형태소 분석에 꼭 포함하고 싶은 단어를 추가한다. 기본적으로는 hashtag들을 noun으로 추가한다. 여기서 해쉬태그 중에 포함하고 싶지 않은 것을 빼는 것을 추천한다.

In [7]:
intake.hashtag_splited = intake.hashtags_split(intake.raw_data[:, 3])
print(np.shape(intake.hashtag_splited))
print(intake.hashtag_splited[0])

(5235,)
['아침식사', '죽', '모닝죽', '단호박', '인테이크', '맛스타그램', '먹스타그램', '식사대용', '다이어트', '두통', '좋아요반사', '맛집', '간편식', '아점', '아픔', '선팔은맞팔', '맛스타', '선팔', '맞팔', '먹스타', '음식', '푸드스타그램', '좋반']


In [8]:
labnosh.hashtag_splited = labnosh.hashtags_split(labnosh.raw_data[:, 3])
print(np.shape(labnosh.hashtag_splited))
print(labnosh.hashtag_splited[0])

(7967,)
['와이식단', '와이_받았다그램', '에잇템', '푸드바', '푸드바', '프로틴바', '푸드바', '푸드바', '에잇템', '랩노쉬', '초코바', '프로틴', '다이어트', '식단', '다이어트식단', '식단관리', '건강식', '다이어트그램', '다이어터그램', '다이어트계정', '다이어트소통', '다이어트기록']


In [10]:
labnosh.raw_data[:2]

array([['user_id', '2018-05-03T23:42:16.000Z',
        'y_diet__ 20180504 오늘의 아침�\n 와이식단  와이_받았다그램\n✔️ 에잇템  푸드바\n✔️우유 한 잔\n에잇템 이벤트로 받게 된 신제품‼️  푸드바 ��\n일단 총평은! 요즘 말로 "갓 띵 템 ��\u200d♀️" 일단 다이어트하면서 초코 좋아하시는분들이면 당장 사주세요! 브라우니 먹는 느낌, 아니면 시중에 파는 초코바 먹는 느낌! 너무 꾸덕, 단단하지도 않으면서 막 촉촉하지도 않은 중간 정도의 식감에 중간 중간 초콜릿이 박혀있고, 코코넛까지 들어있어서 식감 최고!��\n95% 이상이 유기농 원료로 만들어졌기 때문에 믿고 안심하고 먹을 수 있고, 200kcal에 식이섬유가11g (44%), 단백질 13g(24%) 등 영양성분도 최고..!��\n당류는 9g(9%) 정도인데, 개인적으로 단 걸 좋아하는 편이 아니라서 푸드바를 다 먹어갈때쯤 달아서 조금 물린다(?)는 느낌을 받았지만! 그럴 땐 우유 한 모금 마셔주면 다시 홀린듯이 찾게 되는 마성의 매력을 가진 푸드바✨\n또다른 신제품 푸드쉐이크도 같이 보내주셨길래 감동받아서 지기님께 디엠 보냈더니 넘 스윗한 답장 보내주셔서 심쿵❤️\n아무튼 이거 진짜 최고  프로틴바  푸드바 ‼️ 다 먹으면 무조건 재구매 할거에요!! 초코 좋아하시는 다이어터라면 꼭 드셔보세요 ! 후회하지 않으실거에요�\n간단한 한 끼 식사로, 혹은 오후에 당 떨어질 때 간식으로 먹으면 딱 좋을 아이템입니다! 이런 좋은 제품 만들어 준 에잇템에게 감사를!!��❤️',
        '20180504/와이식단/와이_받았다그램/에잇템/푸드바/푸드바/프로틴바/푸드바/푸드바/에잇템/랩노쉬/초코바/프로틴/다이어트/식단/다이어트식단/식단관리/foodbar/diet/eatclean/건강식/다이어트그램/다이어터그램/다이어트계정/다이어트소통/다이어트기록/bbin_d/young.luvv/im_sooki/juning_diet/__diet_bin/h.ji

## intake.add_dic_keyword() 함수는 1차원 list를 받는다.
## 데이터의 형태는 ['단어', '단어', '단어'] or [('단어', Tag), ('단어', Tag)]
## Tag는 'Noun', 'Verb', 'Adjective' 등이 있다.

In [8]:
intake.hashtag_merged = intake.merge_list(intake.hashtag_splited)

intake.add_keyword_dic(intake.hashtag_splited)

In [9]:
labnosh.hashtag_merged = labnosh.merge_list(labnosh.hashtag_splited)

labnosh.add_keyword_dic(labnosh.hashtag_splited)

## 형태소 분석을 진행한다. 한글자는 제외시키는데 혹시 필요하다고 판단되는 글자는 exception_list 파라미터에 리스트로 넣어주면 된다.

In [ ]:
intake.morph_list, intake.nav_list, intake.noun_list, intake.adj_list, intake.verb_list= intake.morph_pos(intake.raw_data[:, 2], exception_list=['맛', '밥', '물', '몸', '죽'])

In [ ]:
labnosh.morph_list, labnosh.nav_list, labnosh.noun_list, labnosh.adj_list, labnosh.verb_list= labnosh.morph_pos(labnosh.raw_data[:, 2], exception_list=['맛', '밥', '물', '몸', '죽'])

## 빈도분석에는 merge된 list가 필요하다.

In [12]:
intake.nav_merged = intake.merge_list(intake.nav_list)

intake.nav_frequency = frequency(intake.nav_merged)

print(intake.nav_frequency [:5])

[('테이크', 6618), ('다이어트', 4639), ('그램', 4087), ('스타', 2790), ('죽', 2511)]


In [13]:
labnosh.nav_merged = labnosh.merge_list(labnosh.nav_list)

labnosh.nav_frequency = frequency(labnosh.nav_merged)

print(labnosh.nav_frequency [:5])

[('랩노쉬', 10886), ('다이어트', 2730), ('맛', 1694), ('미래형식사', 1628), ('올리브영', 1355)]


## 여기서 내가 형태소분석시에 안짤리게 할 단어와 제외하고 싶은 단어, 대체하고 싶은 단어를 선택한다.
## 안짤리게 할 단어는 다시 위로 돌아가 추가한다.
## 대체하고 싶은 단어와 제외하고 싶은 단어는 아래 방법으로 처리한다. 일단 생략한다.

## 분석에서 제외할 단어를 리스트로 넣어준다. 인자로 넣어주는 값은 변하지 않으므로 넣어보고 마음에 안들면 다시 intake.nav_list 로 접근하면 삭제 전 파일로 돌아갈 수 있다.
## 진짜 후회없을 것 같을 때, intake.nav_list에 업데이트 하도록 하자.

In [15]:
intake.nav_frequency = frequency(intake.nav_merged)

In [16]:
intake.nav_frequency [:5]

[('인테이크', 5727), ('다이어트', 2167), ('밀스', 1490), ('아침', 1273), ('모닝죽', 1212)]

In [17]:
modified_nav_list = intake.word_delete(intake.nav_list, ['요거', '여러', '굳이'])
print(modified_nav_list[0])

['아침식사', '죽', '모닝죽', '단호박', '인테이크', '모닝죽', '아침', '기도', '입맛', '하나', '아침', '뚝딱', '맛은', '종류', '맛있는데', '단호박이', '꿀고구마']


## 이번에는 단어를 대체하는 것, 리스트 안에 딕셔너리 형식
#### [{'main': '최종적으로 대체하고 싶은 단어', 'sub_words':['대체할 단어', '대체할 단어']},
####  {'main': '최종적으로 대체하고 싶은 단어', 'sub_words':['대체할 단어', '대체할 단어']}]

In [18]:
modified_nav_list_2 = intake.word_substitute(modified_nav_list, [{'main':'모닝죽', 'sub_words':['단호박', '단호박이', '꿀고구마']}])
print(modified_nav_list_2[0])

['아침식사', '죽', '모닝죽', '모닝죽', '인테이크', '모닝죽', '아침', '기도', '입맛', '하나', '아침', '뚝딱', '맛은', '종류', '맛있는데', '모닝죽', '모닝죽']


### 토픽모델링은 조인된 것이 필요하다.
### join_list 를 통해서 만들기

In [23]:
intake.morph_joined = intake.join_list(intake.nav_list)

In [24]:
labnosh.morph_joined = labnosh.join_list(labnosh.nav_list)

### 파라미터는 다음과 같다. 
#### self.make_lda(self, morph_joined, ntopic=10, learning_method='batch', max_iter=25, random_state=0, n_words=20)
#### ntopic과 n_words를 넣어주면 몇개의 토픽으로 나눌지, 몇개의 단어를 보여줄지를 정할 수 있다.

In [60]:
intake.LDA = SB_LDA()
intake.LDA.make_lda(intake.morph_joined, ntopic=10)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
모닝죽           힘내            있는            다이어트          건강            
아침            운동            많은            저녁            다이어트          
일상            시켜            입니            다이어터          구매            
먹스타그램         않아            있어            점심            먹스타그램         
다이어트          먹겠            같아            들어오시          맛스타그램         
단호박           타서            이벤트           다이어트그램        링크            
간식            주문했           씨씨앙           1개            데일리           
고구마           않는            아니            먹고            유산균           
데일리           씹어            괜찮            받은            건강식           
맛스타그램         드시            없는            사과            후기            
소통            찍어            있다            함께하는          선물            
단호박죽          정모            참여            고구마           간식            
식단조절  

In [ ]:
labnosh.LDA = SB_LDA()
labnosh.LDA.make_lda(labnosh.morph_joined, ntopic=10)

### 그 토픽과 관련된 문서 10개를 보여준다. 두번째 인자는 토픽의 인덱스이고 세번째 인덱스는 보여줄 포스팅의 개수이다.

In [ ]:
topic0 = intake.LDA.related_doc(intake.raw_data[:,2], 0, 4)

In [ ]:
topic0 = labnosh.LDA.related_doc(labnosh.raw_data[:,2], 0, 4)

### 데이터 형식은 같고 할 줄 알겠지?

In [ ]:
intake.SB_word2vec = SB_Word2Vec(intake.nav_list)

intake.flavor = intake.SB_word2vec.get_sim_sen('맛', intake.raw_data[:,2], 4)

In [ ]:
labnosh.SB_word2vec = SB_Word2Vec(labnosh.nav_list)

labnosh.flavor = labnosh.SB_word2vec.get_sim_sen('맛', labnosh.raw_data[:,2], 4)

In [21]:
intake.SB_word2vec.em.wv.most_similar('맛') 

[('맛은', 0.6681041717529297),
 ('식감', 0.6280479431152344),
 ('생각보다', 0.6160725355148315),
 ('새콤달콤', 0.6123064756393433),
 ('미숫가루', 0.5867149829864502),
 ('단맛', 0.5775174498558044),
 ('단점', 0.5737367868423462),
 ('고소', 0.5660829544067383),
 ('조금', 0.5660343170166016),
 ('약간', 0.5656855702400208)]

most_similar(positive=None, negative=None, topn=10, restrict_vocab=None, indexer=None)¶

model.wv.most_similar(positive=['woman', 'king'], negative=['man'])
[('queen', 0.50882536), ...]

model.wv.most_similar_cosmul(positive=['woman', 'king'], negative=['man'])
[('queen', 0.71382287), ...]


model.wv.doesnt_match("breakfast cereal dinner lunch".split())
'cereal'

In [22]:
intake.SB_word2vec.em.wv.most_similar(positive=['아침', '맛'], negative=['운동'])

[('미숫가루', 0.5826406478881836),
 ('생각보다', 0.579865574836731),
 ('심지어', 0.5629949569702148),
 ('귀리', 0.5474948883056641),
 ('완전', 0.5468454957008362),
 ('아주', 0.5345592498779297),
 ('맛나', 0.5310249328613281),
 ('그냥', 0.5308135747909546),
 ('일단', 0.5180487036705017),
 ('좋아', 0.5167618989944458)]

## TFIDF에는 merge 된 리스트를 넣어서 비교해주면 된다. 
## intake만 있어서 패스
## 토픽모델링 된 애들끼리 비교해보는 것도 좋을 듯.

In [23]:
tfidf = SB_Tfidf([intake.nav_merged, labnosh.nav_merged])
tfidf.get_tfidf()
tfidf_of_all = tfidf.tfidf_hangul

for i in tfidf_of_all:
    pprint(i[:10])
    print()

[('흡기', 0.25419146503600865),
 ('튜닝', 0.23128150466795636),
 ('좋은', 0.2160081977559215),
 ('파워젤부스트', 0.18764348491928534),
 ('카스타그램', 0.1800068314632679),
 ('자동차', 0.17346112850096726),
 ('아미노리커버', 0.1680063760323834),
 ('먹으면', 0.16364257405751628),
 ('단팥죽', 0.13418691072716335),
 ('머플러', 0.13091405924601301)]

[('으로', 0.4224923468565074),
 ('쇼콜라', 0.34686938659387423),
 ('미식당', 0.3169824681667258),
 ('많이', 0.22822737708004256),
 ('맛이', 0.19471780187384585),
 ('그래놀라요거트', 0.18384983153670095),
 ('자색고구마', 0.18203850314717682),
 ('푸드쉐이크', 0.17162336490741298),
 ('그린씨리얼', 0.16664221183622158),
 ('함께', 0.13856662179859727)]



In [1]:
labnosh = Social_analysis('labnosh.txt', 'labnosh.official')

meals = Social_analysis('meals.txt', 'intakefoods')

morningjuk = Social_analysis('morningjuk.txt', 'intakefoods')

easy_food = Social_analysis('easy_food.txt')

conven_food = Social_analysis('conven_food.txt')

keywords = [intake, labnosh, meals, morningjuk, easy_food, conven_food]

### 빈도분석

for i in keywords:
    print(len(i.main_text_list), len(i.morph_merged))

labnosh.morph_frequency = analyzer.frequency(labnosh.morph_merged)
pprint(labnosh.morph_frequency)

meals.morph_frequency = analyzer.frequency(meals.morph_merged)
pprint(meals.morph_frequency)

morningjuk.morph_frequency = analyzer.frequency(morningjuk.morph_merged)
pprint(morningjuk.morph_frequency)

easy_food.morph_frequency = analyzer.frequency(easy_food.morph_merged)
pprint(easy_food.morph_frequency)

conven_food.morph_frequency = analyzer.frequency(conven_food.morph_merged)
pprint(conven_food.morph_frequency)

### 토픽모델링

intake.LDA = analyzer.SB_LDA()
intake.LDA.make_lda(intake.morph_joined, ntopic=10)

topic0 = intake.LDA.related_doc(intake.main_text_list, 0)

topic1 = intake.LDA.related_doc(intake.main_text_list, 1)

topic2 = intake.LDA.related_doc(intake.main_text_list, 2)

topic3 = intake.LDA.related_doc(intake.main_text_list, 3)

topic7 = intake.LDA.related_doc(intake.main_text_list, 7)

topic9 = intake.LDA.related_doc(intake.main_text_list, 9)

topic3 = intake.LDA.related_doc(intake.main_text_list, 3)



labnosh.LDA = analyzer.SB_LDA()
labnosh.LDA.make_lda(labnosh.morph_joined, ntopic=10)

meals.LDA = analyzer.SB_LDA()
meals.LDA.make_lda(meals.morph_joined, ntopic=10)

morningjuk.LDA = analyzer.SB_LDA()
morningjuk.LDA.make_lda(morningjuk.morph_joined, ntopic=10)

easy_food.LDA = analyzer.SB_LDA()
easy_food.LDA.make_lda(easy_food.morph_joined, ntopic=10)

conven_food.LDA = analyzer.SB_LDA()
conven_food.LDA.make_lda(conven_food.morph_joined, ntopic=10)

### Word2Vec

intake.SB_word2vec = analyzer.SB_Word2Vec(intake.hashtags_appended)

intake.flavor = intake.SB_word2vec.get_sim_sen('맛', intake.main_hash_dic, intake.main_text_list, 4)


intake.diet = intake.SB_word2vec.get_sim_sen('다이어트', intake.main_hash_dic, intake.main_text_list, 4)


intake.health = intake.SB_word2vec.get_sim_sen('건강', intake.main_hash_dic, intake.main_text_list, 4)


intake.exercise = intake.SB_word2vec.get_sim_sen('운동', intake.main_hash_dic, intake.main_text_list, 4)


intake.morningjuk = intake.SB_word2vec.get_sim_sen('모닝죽', intake.main_hash_dic, intake.main_text_list, 4)


intake.meals = intake.SB_word2vec.get_sim_sen('밀스', intake.main_hash_dic, intake.main_text_list, 4)


intake.gonyak = intake.SB_word2vec.get_sim_sen('곤약젤리', intake.main_hash_dic, intake.main_text_list, 4)




### TFIDF

tfidf = analyzer.SB_Tfidf([intake.morph_merged, labnosh.morph_merged, meals.morph_merged, morningjuk.morph_merged, easy_food.morph_merged, conven_food.morph_merged])

tfidf_of_all = tfidf.tfidf_hangul

for i in tfidf_of_all:
    pprint(i[:10])
    print()

meals_juk_SB_Tfidf = analyzer.SB_Tfidf([meals.morph_merged, morningjuk.morph_merged])

tfidf_meals_juk = meals_juk_SB_Tfidf.tfidf_hangul

for i in tfidf_meals_juk:
    pprint(i[:10])
    print()

intake_meals_SB_Tfidf = analyzer.SB_Tfidf([intake.morph_merged, meals.morph_merged])

tfidf_intake_meals = intake_meals_SB_Tfidf.tfidf_hangul

for i in tfidf_intake_meals:
    pprint(i[:10])
    print()

intake_morningjuk_SB_Tfidf = analyzer.SB_Tfidf([intake.morph_merged, morningjuk.morph_merged])

tfidf_intake_morningjuk = intake_morningjuk_SB_Tfidf.tfidf_hangul

for i in tfidf_intake_morningjuk:
    pprint(i[:10])
    print()

meals_conven_food_SB_Tfidf = analyzer.SB_Tfidf([meals.morph_merged, conven_food.morph_merged])

tfidf_meals_conven_food = meals_conven_food_SB_Tfidf.tfidf_hangul

for i in tfidf_meals_conven_food:
    pprint(i[:10])
    print()

morningjuk_conven_food_SB_Tfidf = analyzer.SB_Tfidf([morningjuk.morph_merged, conven_food.morph_merged])

tfidf_morningjuk_conven_food = morningjuk_conven_food_SB_Tfidf.tfidf_hangul

for i in tfidf_meals_conven_food:
    pprint(i[:10])
    print()

C:\Users\abys\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [9]:
import csv
with open('morph_list.csv', 'w', encoding='utf-8') as f:
    spamwriter = csv.writer(f, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in intake.morph_list:
        spamwriter.writerow(i)

In [16]:
def morph_pos(text_list, add_list, del_list, exception_list = ['맛', '밥', '물', '몸']):
    morph_list = []
    noun_list = []
    adj_list = []
    verb_list = []

    for j in text_list:
        parsed = self.twitter.pos(j)
        temp = []
        n_temp = []
        adj_temp = []
        verb_temp = []

        for i in parsed:
            if self.isHangul(i):
                if not i[0] in del_list:
                    if len(i[0]) > 1:
                        temp.append(i)
                        if i[1] == 'Noun':
                            n_temp.append(i[0])
                        elif i[1] == 'Verb':
                            n_adj.append(i[0])
                        elif i[1] == 'Adjective':
                            n_verb.append(i[0])

                    elif i in exception_list:
                        temp.append(i)
            else: print(i, '한글이 아닙니다.')

        morph_list.append(temp)
        noun_list.append(n_temp)
        adj_list.append(adj_temp)
        verb_list.append(verb_temp)


    nav_list = noun_list + adj_list + verb_list

    return morph_list, nav_list, noun_list, adj_list, verb_list

In [ ]:

    '''def get_from_dic(self, filename):
        self.data = handler.load_by_pickle(filename)
        self.dataset, self.dataset_dic, self.main_text_list, self.morph_list, self.morph_merged, self.morph_joined, self.hashtags_merged, self.hashtags_appended, self.main_hash_dic = handler.create_datasets_with_dic(self.data)

        # dataset, main_text_list, morph_list, morph_merged, morph_joined, hashtags_merged, hashtags_appended, main_hash_dic
    def get_data_from_list(self, filename, add_list, del_list):
        self.data = handler.load_by_pickle(filename)
        self.dataset, self.main_text_list, self.morph_list, self.morph_merged, self.morph_joined, self.hashtags_merged, self.hashtags_appended, self.main_hash_dic = handler.create_datasets_with_list(self.data[1:], add_list, del_list)'''
